In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip insall accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "insall" - maybe you meant "install"


In [ ]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import torch
from transformers import AutoTokenizer
import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/English-Hindi code-mixed parallel corpus.csv')
data.head()

,Sentence,English_Translation
0,@someUSER congratulations on you celebrating b...,@some users congratulate you for celebrating B...
1,@LoKarDi_RT uske liye toh bahot kuch karna pad...,"@Lokardi_ rat we should a lot more for that, b..."
2,@slimswamy yehi to hum semjhane ki koshish kar...,"@Slimswami ehi, this is what i'm expecting you..."
3,@DramebaazKudi cake kaha hai ??,@Where is Dramebajakudi where is the cake?
4,@someUSER i'm in hawaii at the moment . home ...,@some user Don't want to come home next friday...


In [ ]:
val1 = pd.read_csv('/content/drive/MyDrive/st1.csv')
val1.head()

,English,Hindi,Hinglish
0,The average geological temperature of the eart...,1995 से 2004 के दौरान औसत धरातलीय तापमान 1940 ...,from 1995-2004 ke dauran average geology tempe...
1,Dhrudharashtra and Ghandhari 's eldest among t...,धृतराष्ट्र एवं गांधारी के १०० पुत्रों में सबसे...,Dhrudharashtra and Ghandhari ke 100 sons mein ...
2,In this way a major part of the desert in Raja...,इस प्रकार राजस्थान के रेगिस्तान का एक बड़ा भाग...,is prakar rajasthan ke desert ka ek major part...
3,The power of the President to proclaim Ordinan...,राष्ट्रपति की अध्यादेश जारी करने की शक्ति पे न...,President ki ordinance jari karne ki power pr ...
4,More than 1000 elephants were used during the ...,1000 से अधिक हाथी निर्माण के दौरान यातायात हेत...,more than 1000 elephants construction ke daura...


In [ ]:
val2 = pd.read_csv('/content/drive/MyDrive/st2.csv')
val2.head()

,Hinglish,English
0,Abe bigg boss show me baki contestants bhi hai...,Now there are other contestants in the Bigg Bo...
1,Filhal toh behanchod zindagi rape kr rae hai.,At present the behanchod life is raping.
2,agar tumhare sath rape ho jaye to tumhare fath...,What will your father do if Rape happens to you?
3,"I hate it, hostel me banti thi. Us din chawal ...",I hate it. was made in hostel. that time i was...
4,"Tumhare to terrorism failane aate Hain, Indian...","You come to spread terrorism, the Indian Army ..."


In [ ]:
val1.shape, val2.shape

((500, 3), (500, 2))

In [ ]:
val_hinglish1 = val1.Hinglish
val_hinglish2 = val2.Hinglish
val_hinglish = val_hinglish1.append(val_hinglish2)

In [ ]:
len(val_hinglish)

1000

In [ ]:
val_english1 = val1.English
val_english2 = val2.English
val_english = val_english1.append(val_english2)

In [ ]:
len(val_english)

1000

In [ ]:
val = pd.DataFrame()
val['Sentence'] = val_hinglish
val['English_Translation'] = val_english

In [ ]:
val

,Sentence,English_Translation
0,from 1995-2004 ke dauran average geology tempe...,The average geological temperature of the eart...
1,Dhrudharashtra and Ghandhari ke 100 sons mein ...,Dhrudharashtra and Ghandhari 's eldest among t...
2,is prakar rajasthan ke desert ka ek major part...,In this way a major part of the desert in Raja...
3,President ki ordinance jari karne ki power pr ...,The power of the President to proclaim Ordinan...
4,more than 1000 elephants construction ke daura...,More than 1000 elephants were used during the ...
...,...,...
495,Women ke sath rape kab rukega ..........ab to ...,When will the rape with women stop. Now Baba a...
496,Gauahar k FB pe Hina se kam followers hain .. ...,"In gauhar's fb page, less followers than hina...."
497,Pakistan sirf terrorism hi failana janta h usk...,Pakistan only knows to spread terrorism. they ...
498,ye to galat ho gya kisi ek taraf reho ni to ba...,"This has gone wrong, if you stay on one side, ..."


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/test_st2.csv')
test.head()

,Hinglish,English
0,Wah Wah kya baat he par viyapam ghotale ki jaa...,What is that thing? But more than 50 murders h...
1,"Kaun hai eh bhai logo iska bhi rape kardo na, ...","Who is this brother rape her too, this woman i..."
2,Hindu to rape karne ke baad bhi rehta hai ek c...,Hindu remains a criminal even after raped..wha...
3,Ek din ayega ki tumahara hi rape hoga aur jam ...,One day will come and you will be raped and wi...
4,Gujarat BJP ke karyakarta ne rape Kiya he vo K...,Gujarat BJP worker has raped. Show that news o...


In [ ]:
test = test.rename(columns = {'Hinglish' : 'Sentence', 'English' : 'English_Translation'})

In [ ]:
test

,Sentence,English_Translation
0,Wah Wah kya baat he par viyapam ghotale ki jaa...,What is that thing? But more than 50 murders h...
1,"Kaun hai eh bhai logo iska bhi rape kardo na, ...","Who is this brother rape her too, this woman i..."
2,Hindu to rape karne ke baad bhi rehta hai ek c...,Hindu remains a criminal even after raped..wha...
3,Ek din ayega ki tumahara hi rape hoga aur jam ...,One day will come and you will be raped and wi...
4,Gujarat BJP ke karyakarta ne rape Kiya he vo K...,Gujarat BJP worker has raped. Show that news o...
...,...,...
1495,kab se? jab se muslamano ne Hindu ka rape and ...,since when? Ever since Muslims raped Hindus a...
1496,Yaha hindu hi hindu ka rape kare tab koi nahi ata,"Here when Hindu rapes a Hindu, then no one comes."
1497,Shozi ghor se dekhain mein ne Hira ki li hai s...,"Shoji, watch carefully, I hhave took the side ..."
1498,Bhai ab kya ravish ka rape krvoage vo bhi ravi...,"Brother, will you rape Ravish now? That too fr..."


In [ ]:
test.shape

(1500, 2)

In [ ]:
data.shape

(13738, 2)

In [ ]:
(13738/100)*80

10990.4

In [ ]:
# train_rows = data.iloc[:10990]
# train_rows

In [ ]:
# train_rows.shape

In [ ]:
# for i in range(train_rows.shape[0]):
#   print(i)
#   print(train_rows.iloc[i, :]['Sentence'])

In [ ]:
# test_rows = data.iloc[10990:]
# test_rows

In [ ]:
train_rows = data
val_rows = val
test_rows = test

In [ ]:
def load_val():
    d = []
    for i in range(0, val_rows.shape[0]):
      src = val_rows.iloc[i, :]['Sentence']
      tgt = val_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

val_set = load_val()

SRC:  from 1995-2004 ke dauran average geology temperature of earth 1940 se 1980 tak ke ausat temperature se different hai.
TRG:  The average geological temperature of the earth from 1995-2004 is different than that of 1940-1980 .
SRC:  Dhrudharashtra and Ghandhari ke 100 sons mein sabse bade.
TRG:  Dhrudharashtra and Ghandhari 's eldest among their 200 sons .
SRC:  is prakar rajasthan ke desert ka ek major part harvesting and fertile land mein badal jayega.
TRG:  In this way a major part of the desert in Rajasthan would become a harvesting and fertile land .
SRC:  President ki ordinance jari karne ki power pr niyantran.
TRG:  The power of the President to proclaim Ordinance is subject to :
SRC:  more than 1000 elephants construction ke dauran transportation hetu prayog hue the.
TRG:  More than 1000 elephants were used during the time of construction for transportation .
SRC:  Except these , Parsi , Jain n isahi dharma ke mul thinking ko bhi sammilit kiya.
TRG:  Except these , Parsi , 

In [ ]:
def load_train():
    d = []
    for i in range(0, train_rows.shape[0]):
      src = train_rows.iloc[i, :]['Sentence']
      tgt = train_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

train_set = load_train()

Streaming output truncated to the last 5000 lines.
TRG:  lord krishna is also known as butter thief, lord varda is known as house theif
SRC:  @docsreet21 tumko sab pata hai behen
TRG:  @docsreet21 you are aware of all the things
SRC:  mjhe to lgta hai they hate hindu ' too tabhi unki hindu college walo se banti nahi hai :-p
TRG:  i think that they hate hindu people that's why they do not have good relation with hindu college
SRC:  A bhi jao mr.salman kaha ho
TRG:  come where ever you are mr. salman
SRC:  anyway ,  sorry for these random tweets about the finder .  unfollow for the night .  i'll be back to normal sports stuff tomorrow
TRG:  anyway , sorry for these random tweets about the finder .  unfollow for the night .  i will be back to normal sports stuff tomorrow
SRC:  sahi waqt hai netaji, chhaap do ek-do mahine mein. Public ko chahiye aisa kuch.
TRG:  hey minister this is the right print all the stuff, the public wants such kind of things
SRC:  Ya rabba de de koi ____  bhi agar,

In [ ]:
def load_test():
    d = []
    for i in range(0, test_rows.shape[0]):
      src = test_rows.iloc[i, :]['Sentence']
      tgt = test_rows.iloc[i, :]['English_Translation']
      print("SRC: ", src)
      print("TRG: ", tgt)
      d.append(
              {
                  "translation": {
                      "hinglish": src.strip(),
                      "english": tgt.strip()
                  }
              }
      )
    print(f'total size of data is {len(d)}')
    tdata = pd.DataFrame(d)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    return dataset

test_set = load_test()

SRC:  Wah Wah kya baat he par viyapam ghotale ki jaaz me 50 se jyada murder ho Gaye he . Apne tou world record Bana diya Saab. Badhai ho apko
TRG:  What is that thing? But more than 50 murders have taken place in the investigation of Viapam scam. You have made a world record, Saab. Congratulations to you
SRC:  Kaun hai eh bhai logo iska bhi rape kardo na, Bahut bechen hai eh mahila, shayed  peechlee janam ki esi icchaa hai iski.BJP k time do it https://twitter.com/SahilThoughts/status/892906767683903492 …
TRG:  Who is this brother rape her too, this woman is very restless, maybe this is the desire of her last life. Do it in BJP's time. https://twitter.com/SahilThoughts/status/892906767683903492 …
SRC:  Hindu to rape karne ke baad bhi rehta hai ek criminal..What is ur point???
TRG:  Hindu remains a criminal even after raped..what is ur point???

SRC:  Ek din ayega ki tumahara hi rape hoga aur jam kad rape hoga
TRG:  One day will come and you will be raped and will be forced raped.
SRC: 

In [ ]:
test_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 1500
})

In [ ]:
val_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 1000
})

In [ ]:
train_set

Dataset({
    features: ['id', 'translation'],
    num_rows: 13738
})

In [ ]:
#!pip install sentencepiece

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[250004, 35378, 4, 903, 1632, 149357, 38, 2], [250004, 3293, 83, 15700, 149357, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "hinglish"
target_lang = "english"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train_set = train_set.map(preprocess_function, batched=True)
tokenized_val_set = val_set.map(preprocess_function, batched=True)
tokenized_test_set = test_set.map(preprocess_function, batched=True)

Map:   0%|          | 0/13738 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
tokenized_train_set['translation']

[{'english': "@some users congratulate you for celebrating British kid singers Sophia Grace's and Rosie's 1st anniversary visit of your show",
  'hinglish': "@someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how"},
 {'english': '@Lokardi_ rat we should a lot more for that, by this evi people nothing will happen #ForTheSakeOfHumanity',
  'hinglish': '@LoKarDi_RT uske liye toh bahot kuch karna padega ye pappiyon se kaam nahi chalega #ForTheSakeOfHumanity'},
 {'english': "@Slimswami ehi, this is what i'm expecting you to understand, people invest everything in this isnt it.",
  'hinglish': '@slimswamy yehi to hum semjhane ki koshish kar rahe hain. Log to sab kuch ko issi mein tol dete hain. Context:http://www.opindia.com/2015/02/buzzfeed-editor-tries-to-make-political-tweets-you-wont-believe-what-happened-next/\xa0…'},
 {'english': '@Where is Dramebajakudi where is the cake?',
  'hinglish': '@DramebaazKud

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

<ipython-input-38-b5f524766ed4>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from accelerate import Accelerator
accelerator = Accelerator()

In [ ]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_810/",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=6,
                        per_device_eval_batch_size=6,
                        learning_rate=5e-5,
                        num_train_epochs=3,
                        predict_with_generate=True,
                        logging_dir="/logs",
                        logging_steps=10000,
                        save_steps=10000,
                        report_to="none"
                        )

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
trainer = Seq2SeqTrainer(model=model,
                args=args,
                data_collator=data_collator,
                train_dataset=tokenized_train_set,
                eval_dataset=tokenized_val_set,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
                )

tokenized_datasets, tokenized_test_set, trainer2 = accelerator.prepare(
     tokenized_train_set, tokenized_val_set, trainer
      )

trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: translation, id. If translation, id are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13738
  Num Epochs = 3
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 6870
  Number of trainable parameters = 610879488
You're using a MBart50TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is f

Epoch,Training Loss,Validation Loss


In [ ]:
# trainer

In [ ]:
#torch.save(model, '/content/mbart_810/model.pt')

In [ ]:
#model_dir = '/content/drive/MyDrive/801_baseline_mbart'
# trainer.save_model(model_dir + 'baseline1')

In [ ]:
# retreive the saved model
#model_retrieved = MBartForConditionalGeneration.from_pretrained(model_dir + 'baseline1',
#                                                        local_files_only=True)

In [ ]:
#model_retrieved

In [ ]:
#trainer = model_retrieved

In [ ]:
predictions = trainer.predict(tokenized_test_set)

In [ ]:
predictions.predictions

In [ ]:
import pickle as pkl

In [ ]:
with open('/content/drive/MyDrive/801_test_predictions_list.pkl', 'wb') as f:
  pkl.dump(predictions.predictions, f)

In [ ]:
predictions.label_ids

In [ ]:
predictions.metrics

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(predictions.predictions[0])

In [ ]:
for i in range(0, len(predictions.predictions)):
  print("Test data point number ", i)
  print("Source English: ")
  print(tokenized_test_set[i]['translation']['english'])
  print("Source Hinglish: ")
  print(tokenized_test_set[i]['translation']['hinglish'])
  print("Translation: ")
  print(tokenizer.decode(predictions.predictions[i]))
  print()

In [ ]:
x_test = []
y_test = []
y_pred = []
for i in range(0, len(predictions.predictions)):
  x_test.append(tokenized_test_set[i]['translation']['english'])
  y_test.append(tokenized_test_set[i]['translation']['hinglish'])
  y_pred.append(tokenizer.decode(predictions.predictions[i]))

In [ ]:
len(x_test), len(y_test), len(y_pred)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
bleu = []
# reference and predicted sentences
for i in range(0, 1000):
  ref_sent = y_test[i].split()
  pred_sent = y_pred[i].split()
  # calculate BLEU score
  score = sentence_bleu(ref_sent, pred_sent)
  bleu.append(score)

print(bleu)

In [ ]:
import sacrebleu
ref = ['This is a test .']
hyp = ['This is a train .']
bleu = sacrebleu.corpus_bleu(hyp, [ref])
print(bleu.score)

In [ ]:
import sacrebleu
bleu = []
# reference and predicted sentences
for i in range(0, 1000):
  ref_sent = [y_test[i]]
  pred_sent = y_pred[i]
  # calculate BLEU score
  score = sacrebleu.corpus_bleu(ref_sent, pred_sent)
  bleu.append(score.score)

print(bleu)

In [ ]:
np.sum(bleu)/len(bleu)